In [ ]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from bs4 import BeautifulSoup
import os
from datetime import datetime
import time

def file_config(start_url, i=2):
    # Configuration du driver
    driver_path = r"/usr/local/bin/chromedriver"
    options = webdriver.ChromeOptions()
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    options.add_argument('--headless')
    driver = webdriver.Chrome(executable_path=driver_path, options=options)
    driver.set_page_load_timeout(6000)
    attempts = i
    while attempts >0:
        try :
            driver.get(start_url)
            driver.maximize_window()
            attempts = 0
        except TimeoutException:
            attempts = attempts - 1
    return driver

In [8]:

driver = file_config(start_url = r'https://annonces.abidjan.net/immobiliers?search_immo_form%5BidTypeImmobilier%5D=&search_immo_form%5Bprix%5D=&search_immo_form%5Bsuperficie%5D=&search_immo_form%5Bstatut%5D=Location&search_immo_form%5Bville%5D=&page=1')

wait = WebDriverWait(driver, 10000)

data = []

def scrape_data(cards, immo_name):

    links = []
    for card in cards:
        link = card.find_element(By.CSS_SELECTOR, 'a').get_attribute('href')
        links.append(link)

    for link in links:
        attempts = 3
        while attempts >0:
            try :
                driver.get(link)
                driver.refresh()
                attempts = 0
            except TimeoutException:
                attempts = attempts - 1
        # try:
        try:
            wait.until(EC.visibility_of_element_located((By.CSS_SELECTOR, 'body > div.content.reset-top-content > div.sub-content > section.section-ad-right > div > div > div.annonce-detail')))
        except TimeoutException:
            continue
        # Utilisation de BeautifulSoup pour récupérer les détails de l'annonce
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        try :
            details = soup.select_one('body > div.content.reset-top-content > div.sub-content > section.section-ad-right > div > div > div.annonce-detail')
        except AttributeError:
            continue

        # from details_group1
        try:
            price = details.select_one("div.annonce-details-price").text.strip().split(": ")[-1].replace('\t', '').replace('\n', '')
            title = soup.select_one("body > div.content.reset-top-content > div.sub-content > section.section-ad-right > div > div > div:nth-child(2) > h1").text.strip().replace('\t', '').replace('\n', '')
            localisation = details.select_one("div:nth-child(4) > div:nth-child(3) > strong").text.strip() + " " + details.select_one("div:nth-child(4) > div:nth-child(4) > strong").text.strip()

        except AttributeError:
            continue

        immo_type = immo_name

        # from details_group2
        try:
            details_content = soup.select_one('body > div.content.reset-top-content > div.sub-content > section.section-ad-right > div > div > div.annonce-detail > div > div:nth-child(4)').text.strip().replace('\t', '').replace('\n', '')

            superfic = "Superficie".lower() in details_content.lower()
            pieces = "Nombre de pièce".lower() in details_content.lower()
            bain =  "Nombre de salle d'eau".lower() in details_content.lower()

            if pieces and bain and superfic:
                superficie = f"{details.select_one('div:nth-child(4) > div:nth-child(9) > strong').text.strip()} m2"
                nb_pieces = details.select_one('div:nth-child(4) > div:nth-child(7) > strong').text.strip()
                nb_salle_de_bain = details.select_one('div:nth-child(4) > div:nth-child(8) > strong').text.strip()
            elif pieces and bain and not superfic:
                superficie = None
                nb_pieces = details.select_one('div:nth-child(4) > div:nth-child(7) > strong').text.strip()
                nb_salle_de_bain = details.select_one('div:nth-child(4) > div:nth-child(8) > strong').text.strip()
            elif pieces and superfic and not bain:
                superficie = details.select_one('div:nth-child(4) > div:nth-child(8) > strong').text.strip()
                nb_pieces = details.select_one('div:nth-child(4) > div:nth-child(7) > strong').text.strip()
                nb_salle_de_bain = None
            elif bain and superfic and not pieces:
                superficie = details.select_one('div:nth-child(4) > div:nth-child(8) > strong').text.strip()
                nb_pieces = None
                nb_salle_de_bain = details.select_one('div:nth-child(4) > div:nth-child(7) > strong').text.strip()
            elif pieces and not superfic and not bain:
                superficie = None
                nb_pieces = details.select_one('div:nth-child(4) > div:nth-child(7) > strong').text.strip()
                nb_salle_de_bain = None
            elif superfic and not pieces and not bain:
                superficie = details.select_one('div:nth-child(4) > div:nth-child(7) > strong').text.strip()
                nb_pieces = None
                nb_salle_de_bain = None
            elif bain and not pieces and not superfic:
                superficie = None
                nb_pieces = None
                nb_salle_de_bain = details.select_one('div:nth-child(4) > div:nth-child(7) > strong').text.strip()
            else:
                superficie = None
                nb_pieces = None
                nb_salle_de_bain = None
        except AttributeError:
            continue


        # from details_group3
        try:
            description = soup.select('body > div.content.reset-top-content > div.sub-content > section.section-ad-right > div > div > div.annonce-detail > div > div.annonce-p-subtitle.anim')[-1].text.strip().replace('\n', ' ').replace('\t', '')
        except AttributeError:
            description = None

        try:
            annonceur = soup.select_one('div.annonce-p-wrapper.anim > div.annonce-p-detail > div.annonce-p-name').text.strip().replace('\n', ' ').replace('\t', ' ')
        except AttributeError:
            annonceur = None
        current_datetime = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

        try :
            data.append({
                'title': title,
                'price': price,
                'localisation': localisation,
                'superficie': superficie,
                "type d'immobilier": immo_type,
                'nb_pieces': nb_pieces,
                "nb_salle_de_bain": nb_salle_de_bain,
                'scraping_date': current_datetime,
                "annonceur" : annonceur,
                "link" : link,
                'description': description
            })
        except :
          continue
        print(data[-1])
    return data

i = 1

# Dictionnaire des types d'immobilier et de leurs valeurs
type_immo_dict = {
    '1': 'Appartements',
    '6': 'Villas',
    '12': 'Maisons',
}

current_length = 0

for immo_id, immo_name in type_immo_dict.items():
    url = f'https://annonces.abidjan.net/immobiliers?search_immo_form%5BidTypeImmobilier%5D={immo_id}&search_immo_form%5Bprix%5D=&search_immo_form%5Bsuperficie%5D=&search_immo_form%5Bstatut%5D=Location&search_immo_form%5Bville%5D=&page=1'
    driver.get(url)
    driver.refresh()

    while True:
        
        try:
            # Attendre que les éléments soient visibles
            cards_presence = wait.until(EC.visibility_of_element_located((By.CSS_SELECTOR, 'body > div.content.reset-top-content > div.sub-content > section.section-ad-right')))
            # Créer une liste de cartes
            cards = cards_presence.find_elements(By.CSS_SELECTOR, 'div > div.grid3 > div > div')

            time.sleep(10)

            pagination = wait.until(EC.visibility_of_element_located((By.CSS_SELECTOR, 'ul.pagination')))

            # Récupérer tous les éléments <li> dans la pagination
            page_items = pagination.find_elements(By.CSS_SELECTOR, 'li.page-item')

            if len(data) > 1000:
                break


            # Vérifier si le dernier élément a la classe 'disabled'
            if 'disabled' in page_items[-1].get_attribute('class'):
                print("Dernier élément désactivé, fin de la pagination.")

                scrape_data(cards, immo_name)
                i+=1
                break

            else:

                print(f"Il y a encore des pages disponibles pour {immo_name}.")
                next_link_presence = wait.until(EC.visibility_of_element_located((By.CSS_SELECTOR, 'body > div.content.reset-top-content > div.sub-content > section.section-ad-right > div > div.Zebra_Pagination > nav > ul')))
                next_link = next_link_presence.find_elements(By.CSS_SELECTOR, 'li > a')
                start_url = next_link[-1].get_attribute('href')

                scrape_data(cards, immo_name)
                driver.get(start_url)

                i += 1

        except:
            print("Une erreur s'est produite lors de la collecte sur Abidjan.net")
            break

print("fin du scraping")


driver.quit()


In [ ]:
data = pd.DataFrame(data)
data

import locale

# Définir la locale en français
locale.setlocale(locale.LC_TIME, 'fr_FR.UTF-8')

# Obtenir la date et l'heure actuelles
current_datetime = datetime.now()
formatted_date = current_datetime.strftime("%Y-%m-%d %H:%M:%S")
formatted_month = current_datetime.strftime("%B")
formatted_day = current_datetime.strftime("%d")

# Déterminer le système d'exploitation
import platform
if platform.system() == 'Windows':
    dynamic_path = f'D:\\Bureau\\MemoiresStages\\Travaux_techniques\\Scrapping\\Datasets\\{formatted_month}\\{formatted_day}_{formatted_month}\\AbidjanNet_{formatted_day}_{formatted_month}.csv'
else:
    dynamic_path = f'/mnt/d/Bureau/MemoiresStages/Travaux_techniques/Scrapping/Datasets/{formatted_month}/{formatted_day}_{formatted_month}/AbidjanNet_{formatted_day}_{formatted_month}.csv'

os.makedirs(os.path.dirname(dynamic_path), exist_ok=True)

data.to_csv(dynamic_path, index=False)

